In [14]:
import pandas as pd

# loading train data
def load_data(filename):
    return pd.read_csv(filename)

train_data = load_data("train.csv") 
test_data = load_data("test.csv") 


data visualizing

In [7]:
# train_data.Sex
train_data.Sex[1]

'female'

feature engineering

In [16]:
# embarked 
train_data["Embarked"].fillna(value='S', inplace=True)
test_data['Fare'].fillna(value=train_data.Fare.mean(), inplace=True)

In [10]:
# age
mean_age_miss = round(train_data[train_data["Name"].str.contains("Miss.",na=False)]['Age'].mean())
mean_age_mrs = round(train_data[train_data["Name"].str.contains("Mrs.",na=False)]['Age'].mean())
mean_age_mr = round(train_data[train_data["Name"].str.contains("Mr.",na=False)]['Age'].mean())
mean_age_master = round(train_data[train_data["Name"].str.contains("Master.",na=False)]['Age'].mean())

In [11]:
def fill_age(name_age):
    name= name_age[0]
    age = name_age[1]
    
    if pd.isnull(age):
        if "Mr." in name:
            return mean_age_mrs
        if "Mrs." in name:
            return mean_age_mrs
        if "Miss." in name:
            return mean_age_miss
        if "Master." in name:
            return mean_age_master
        if "Dr." in name:
            return mean_age_master
        if "Ms." in name:
            return mean_age_miss
    else:
        return age
    

In [17]:
train_data["Age"] = train_data[["Name", "Age"]].apply(fill_age, axis= 1)
test_data["Age"] = train_data[["Name", "Age"]].apply(fill_age, axis= 1)

In [20]:
# <!-- new feature  -->
import numpy as np

bins =[0, 20, 40, np.inf]
names = ["20-", "20-40", "40+"]
train_data["AgeRange"] = pd.cut(train_data.Age, bins, labels=names)
test_data["AgeRange"] = pd.cut(test_data.Age, bins, labels=names)

In [28]:
# new feature deck and room
def deck_room(data):
    data["Deck"] = data["Cabin"].str.slice(0, 1)
    data["Room"] = data["Cabin"].str.slice(1, 5).str.extract("([0-9]+)",
                                                            expand=False, 
                                                           ).astype("float")
    data["Deck"] = data["Deck"].fillna("N")
    data["Room"] = data["Room"].fillna(data["Room"].mean())
    return data

In [29]:
train_data = deck_room(train_data)
test_data = deck_room(test_data)

In [30]:
# family size 
def family_size(data):
    data["Family_size"] = data["SibSp"] + data["Parch"] + 1
    return data
train_data = family_size(train_data)
test_data = family_size(test_data)

In [32]:
def create_alone_feature(sibpch):
    if(sibpch[0]+sibpch[1]) == 0:
        return 1
    else:
        return 0

In [34]:
# new feature alone
train_data["Alone"] = train_data[["SibSp", "Parch"]].apply(create_alone_feature, axis = 1)
train_data["Alone"] = test_data[["SibSp", "Parch"]].apply(create_alone_feature, axis = 1)

feature correlation

In [36]:
df  = train_data

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
# corr_matrix = df.corr()
# cmap = sns.diverging_palette(230, 20, as_cmap = True)
# sns.heatmap(corr_matrix, annot=None, cmap = cmap)

final feature engineering

In [41]:
features = ["Pclass", "Sex", "Family_size", "Embarked", "Fare",
            "Age", "AgeRange",]
label = "Survived"

In [44]:
# test_data[features].head()

In [45]:
# sex categories
train_data["Sex"] = train_data["Sex"].map({"female":1, "male":2})
test_data["Sex"] = test_data["Sex"].map({"female":1, "male":2})

In [46]:
# Embarked Categories
categories ={ "S":1, "C": 2, "Q":3}
train_data["Embarked"] = train_data["Embarked"].map(categories)
test_data["Embarked"] = test_data["Embarked"].map(categories)

In [47]:
# deck categories
d_categories = {"C":1, "B":2, "D":3, "E":4, "A":5}
train_data["Deck"] = train_data["Deck"].map(d_categories)
test_data["Deck"] = test_data["Deck"].map(d_categories)

In [48]:
# AgeRange
ar_categories = {"20-":1, "20-40":2, "40+":3}
train_data["AgeRange"] = train_data["AgeRange"].map(ar_categories)
test_data["AgeRange"] = test_data["AgeRange"].map(ar_categories)


In [49]:
from sklearn.model_selection import train_test_split

X, y = train_data[features], train_data[label].values
# spliting 30 -70
X_train , X_test, y_train, y_test = train_test_split(X, y, 
                                                     test_size= 0.30, 
                                                      random_state= 0)

In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [52]:
# scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.fit_transform(X_test)
X_eval = sc.fit_transform(test_data[features])

In [88]:
X_train[:10]

array([[-1.5325562 ,  0.72592065, -0.54860377, -0.55270799, -0.12253019,
         1.53767522,  1.69612763],
       [-1.5325562 , -1.37756104,  0.04476454,  1.01246055,  0.91812372,
         1.38864996,  1.69612763],
       [ 0.84844757,  0.72592065,  3.60497445, -0.55270799,  0.29950338,
        -2.18795642, -1.60602085],
       [-1.5325562 ,  0.72592065,  0.04476454, -0.55270799,  0.92970216,
         1.76121312,  1.69612763],
       [ 0.84844757, -1.37756104,  0.04476454,  1.01246055, -0.37329694,
         0.41998573,  0.04505339],
       [-0.34205431,  0.72592065,  0.63813286, -0.55270799,  0.08900506,
        -1.66636799, -1.60602085],
       [-1.5325562 ,  0.72592065,  0.04476454,  1.01246055,  4.4601281 ,
        -0.47416586,  0.04505339],
       [ 0.84844757,  0.72592065, -0.54860377, -0.55270799, -0.50619708,
        -0.7722164 , -1.60602085],
       [ 0.84844757,  0.72592065, -0.54860377, -0.55270799, -0.50619708,
        -0.02709007,  0.04505339],
       [ 0.84844757,  0.7259

fit models

# random forest classifier

In [79]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators= 100, max_depth= 4,
                                    random_state=0)
forest_clf = forest_clf.fit(X_train, y_train)

In [81]:
# score 
predictions_train = forest_clf.predict(X_train)
predictions_test = forest_clf.predict(X_test)
print("train score ", accuracy_score(y_train, predictions_train))
print("test score ", accuracy_score(y_test, predictions_test))
print(classification_report(y_test, predictions_test ))

train score  0.8459069020866774
test score  0.8097014925373134
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       168
           1       0.76      0.72      0.74       100

    accuracy                           0.81       268
   macro avg       0.80      0.79      0.79       268
weighted avg       0.81      0.81      0.81       268



In [82]:
# predictions
forest_predictions = forest_clf.predict(X_eval)

In [85]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': forest_predictions})
output.to_csv("forest_mission.csv", index= False )

In [ ]:
# .7707

# Decision tree classifier

In [57]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf = dt_clf.fit(X_train, y_train)

In [86]:
predictions_train = dt_clf.predict(X_train)
predictions_test = dt_clf.predict(X_test)
print("train score ", accuracy_score(y_train, predictions_train))
print("test score ", accuracy_score(y_test, predictions_test))
print(classification_report(y_test, predictions_test ))

train score  0.9775280898876404
test score  0.7873134328358209
              precision    recall  f1-score   support

           0       0.81      0.86      0.84       168
           1       0.74      0.66      0.70       100

    accuracy                           0.79       268
   macro avg       0.78      0.76      0.77       268
weighted avg       0.78      0.79      0.78       268



In [87]:
# predictions
dt_predict = dt_clf.predict(X_eval)
# output
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': dt_predict})
output.to_csv("dt_mission.csv", index= False )


In [ ]:
# 0.703

# SVC

In [75]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf = svc_clf.fit(X_train, y_train)

In [76]:
# score
predictions_train = svc_clf.predict(X_train)
predictions_test = svc_clf.predict(X_test)
print("train score ", accuracy_score(y_train, predictions_train))
print("test score ", accuracy_score(y_test, predictions_test))
print(classification_report(y_test, predictions_test ))

train score  0.8459069020866774
test score  0.8059701492537313
              precision    recall  f1-score   support

           0       0.83      0.88      0.85       168
           1       0.77      0.69      0.73       100

    accuracy                           0.81       268
   macro avg       0.80      0.78      0.79       268
weighted avg       0.80      0.81      0.80       268



In [77]:
# predictions
svc_predicts = svc_clf.predict(X_eval)

In [83]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': svc_predicts})
output.to_csv("svc_mission.csv", index= False )

In [ ]:
#.7631

# Gradient Boosting Classifier

In [62]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators = 100,
                                   learning_rate=0.5,
                                   max_depth= 3, random_state= 140)
gb_clf = gb_clf.fit(X_train, y_train)

In [63]:
#Score
predictions_train = gb_clf.predict(X_train)
predictions_test = gb_clf.predict(X_test)
print('train score :',accuracy_score(y_train ,predictions_train))
print('test score :',accuracy_score(y_test , predictions_test))

train score : 0.971107544141252
test score : 0.8097014925373134


In [ ]:
# 0.72

# logistic Regression

In [64]:
from sklearn.linear_model import LogisticRegression
l_reg = LogisticRegression()
l_reg = l_reg.fit(X_train, y_train)

In [65]:
#Score
predictions_train = l_reg.predict(X_train)
predictions_test = l_reg.predict(X_test)
print('train score :',accuracy_score(y_train ,predictions_train))
print('test score :',accuracy_score(y_test , predictions_test))

train score : 0.7961476725521669
test score : 0.8097014925373134


analysis the result and submit

In [66]:
fmodel = GradientBoostingClassifier(n_estimators=100, 
                                   learning_rate= 0.5, 
                                   max_depth= 3, 
                                   random_state= 140)
fmodel.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.5, random_state=140)

In [67]:
#Score
predictions_train = fmodel.predict(X_train)
predictions_test = fmodel.predict(X_test)
print('train score :',accuracy_score(y_train ,predictions_train))
print('test score :',accuracy_score(y_test , predictions_test))
print(classification_report(y_test, predictions_test ))

train score : 0.971107544141252
test score : 0.8097014925373134
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       168
           1       0.79      0.67      0.72       100

    accuracy                           0.81       268
   macro avg       0.80      0.78      0.79       268
weighted avg       0.81      0.81      0.81       268



In [68]:
# predicts 
predictions = fmodel.predict(X_eval)

# output the result

In [69]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv("s_mission.csv", index= False )

# .7200 result